# 02 - Build Non-ICU RAAS Early Exposure Tables
## 0. Overview

This notebook constructs early renin–angiotensin–aldosterone system (RAAS) inhibitor exposure variables for adult, non-ICU hospital admissions using the MIMIC-IV database.

Using SQL-based preprocessing in BigQuery, inpatient medication prescription records are linked to hospital admission timestamps to identify early-in-admission RAAS inhibitor exposure, defined as at least one prescription for an angiotensin-converting enzyme inhibitor (ACE inhibitor) or angiotensin II receptor blocker (ARB) with a documented start time occurring on or after hospital admission and within the first 24 hours of admission.

The resulting exposure indicators are constructed at the hospital admission (hadm_id) level and materialized as BigQuery tables for use in downstream cohort description and outcome analyses.

## 1. Purpose

The purpose of this notebook is to define and construct early RAAS inhibitor exposure variables at the hospital admission level for adult, non-ICU admissions in MIMIC-IV.

Specifically, this notebook:
- Defines early RAAS inhibitor exposure using inpatient prescription records from the MIMIC-IV prescriptions table
- Restricts exposure ascertainment to prescriptions with start times occurring within 24 hours after hospital admission, prior to outcome occurrence, to preserve temporal ordering
- Constructs admission-level binary indicators for:
  - ACE inhibitor exposure
  - ARB exposure
  - Combined ACE inhibitor and ARB exposure
  - Any RAAS inhibitor exposure
- Materializes these exposure variables as reproducible BigQuery tables for downstream baseline characterization and outcome modeling

This notebook focuses exclusively on exposure variable construction and does not perform descriptive analyses, statistical modeling, or outcome evaluation.

## 2. Data Sources

- **MIMIC-IV v3.1** (BigQuery public dataset)
- Project: `mimic-iv-portfolio`

**Source Tables:**
- `physionet-data.mimiciv_3_1_hosp.prescriptions`<br>
  (BigQuery public dataset)

- `mimic-iv-portfolio.nonicu_raas.nonicu_admissions`<br>
  (created in 01 using `02_exclude_icu_admissions.sql`)

- `mimic-iv-portfolio.nonicu_raas.exposure_raas_early`<br>
  (created in 02 using `03_define_exposure_raas_early.sql`)

**Derived datasets created in this notebook:**
- `mimic-iv-portfolio.nonicu_raas.exposure_raas_early`
- `mimic-iv-portfolio.nonicu_raas.analysis_dataset`

## 3. Build Cohort and Exposure Tables in BigQuery
### 3.1 BigQuery Client Initialization and Helper Functions

In [1]:
# Use Application Default Credentials (my user account)
# This account already has PhysioNet BigQuery access.

from google.cloud import bigquery
from google.auth import default
from pathlib import Path

# 1. Define project ID, dataset, and table references
PROJECT_ID = "mimic-iv-portfolio"
DATASET = "nonicu_raas"

TABLE_NONICU = f"{PROJECT_ID}.{DATASET}.nonicu_admissions"   # Created in 02_exclude_icu_admissions.sql
TABLE_EXPO = f"{PROJECT_ID}.{DATASET}.exposure_raas_early"   # Created in 03_define_exposure_raas_early.sql
TABLE_ANALYSIS = f"{PROJECT_ID}.{DATASET}.analysis_dataset"  # Created in 04_build_analysis_dataset.sql


# 2. Get ADC credentials
creds, adc_project = default()

# 3. Initialize BigQuery client
client = bigquery.Client(project=PROJECT_ID, credentials=creds)

# 5. Helper to run a SQL script file (DDL, CREATE TABLE, etc.)
def run_sql_script(path) :
    """
    Read a .sql file from disk, execute it in BigQuery,
    and wait until the job finishes.
    Use this for CREATE TABLE / INSERT INTO scripts.
    """
    sql_path = Path(path)
    with sql_path.open("r") as f:
        query = f.read()
    job = client.query(query)
    job.result()
    print(f"Executed SQL script: {sql_path.name}")

# 6. Helper for read-only SELECT queries → DataFrame
def query_to_df(query) :
    """
    Run a SELECT query in BigQuery and return a pandas DataFrame.
    """
    job = client.query(query)
    return job.to_dataframe(create_bqstorage_client=False)

### 3.2 Execute SQL to Define Early RAAS Exposure

In [2]:
# Excecute SQL script (03_define_exposure_raas_early.sql)
run_sql_script("../sql/03_define_exposure_raas_early.sql")

Executed SQL script: 03_define_exposure_raas_early.sql


### 3.3 Load and Inspect Early RAAS Exposure Table

In [3]:
q_expo = f"""
SELECT *
FROM `{TABLE_EXPO}`
"""
df_expo = client.query(q_expo).to_dataframe()

df_expo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 460786 entries, 0 to 460785
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype
---  ------           --------------   -----
 0   subject_id       460786 non-null  Int64
 1   hadm_id          460786 non-null  Int64
 2   acei_early       460786 non-null  Int64
 3   arb_early        460786 non-null  Int64
 4   raas_both_early  460786 non-null  Int64
 5   raas_any_early   460786 non-null  Int64
dtypes: Int64(6)
memory usage: 23.7 MB


### 3.4 Build Final Admission-Level Analysis Dataset

In [4]:
# Excecute SQL script (03_build_analysis_dataset.sql)
run_sql_script("../sql/03_build_analysis_dataset.sql")

Executed SQL script: 03_build_analysis_dataset.sql


### 3.5 Load and Inspect Final Analysis Dataset

In [5]:
q_analysis = f"""
SELECT *
FROM `{TABLE_ANALYSIS}`
"""
df = client.query(q_analysis).to_dataframe()

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 460786 entries, 0 to 460785
Data columns (total 25 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   subject_id            460786 non-null  Int64         
 1   hadm_id               460786 non-null  Int64         
 2   admittime             460786 non-null  datetime64[us]
 3   dischtime             460786 non-null  datetime64[us]
 4   deathtime             2324 non-null    datetime64[us]
 5   hospital_expire_flag  460786 non-null  Int64         
 6   admission_type        460786 non-null  object        
 7   admission_location    460785 non-null  object        
 8   discharge_location    311810 non-null  object        
 9   insurance             452862 non-null  object        
 10  language              460377 non-null  object        
 11  marital_status        454118 non-null  object        
 12  race                  460786 non-null  object        
 13 

## 4. Sanity Checks

### 4.1 Exposure Indicator Validation and Prevalence

In [39]:
# Normalize indicator variables to boolean type to ensure consistent logical operations
# regardless of whether the original encoding is 0/1 or boolean.
acei = df_expo["acei_early"].astype(bool)
arb  = df_expo["arb_early"].astype(bool)
any_ = df_expo["raas_any_early"].astype(bool)
both = df_expo["raas_both_early"].astype(bool)


# Validate logical consistency of derived exposure indicators:
# - raas_any_early should be TRUE if either acei_early or arb_early is TRUE
# - raas_both_early should be TRUE if both acei_early and arb_early are TRUE
checks = {
    "any":  ((acei | arb) == any_).mean(),
    "both": ((acei & arb) == both).mean(),
}

# Report the proportion of admissions for which each consistency check is satisfied
for k, v in checks.items():
    print(f"Consistency ({k}): {v:.4f}")

Consistency (any): 1.0000
Consistency (both): 1.0000


In [57]:
# Identify and count any records where logical inconsistencies are present
mask_any  = (acei | arb) != any_
mask_both = (acei & arb) != both

print("Any mismatches:",  mask_any.sum())
print("Both mismatches:", mask_both.sum())

Any mismatches: 0
Both mismatches: 0


**Descriptive Summary**

- Logical consistency checks confirmed perfect agreement between the derived RAAS exposure indicators (raas_any_early and raas_both_early) and their component variables (acei_early and arb_early).

- Specifically, the logical definitions of raas_any_early (derived from acei_early OR arb_early) and raas_both_early (derived from acei_early AND arb_early) were satisfied for 100% of admissions, with no mismatches identified.
	
- These results support the internal validity of the exposure indicator construction.

### 4.2 Row Count Comparison Between Cohort and Exposure Tables

In [59]:
q_counts = f"""
SELECT
  (SELECT COUNT(*) FROM `{TABLE_NONICU}`) AS n_nonicu,
  (SELECT COUNT(*) FROM `{TABLE_EXPO}`) AS n_expo
"""
client.query(q_counts).to_dataframe()

,n_nonicu,n_expo
0,460786,460786


**Descriptive Summary**

- The non-ICU admissions table and the early RAAS exposure table each contained 460,786 records, indicating identical row counts at the hospital admission level.
- This confirms that early RAAS exposure variables were generated for all admissions in the analytic cohort, with no record loss during table construction or joins.

### 4.3 Distribution of Matched RAAS Drugs

In [61]:
df_drugs = query_to_df(
    Path("../sql/04_check_raas_drugs.sql").read_text()
)

df_drugs

,class,drug_lc,n_rows
0,ACEi,lisinopril,37520
1,ARB,losartan potassium,13005
2,ARB,valsartan,6074
3,ACEi,captopril,2149
4,ACEi,enalapril maleate,1871
5,ACEi,quinapril,597
6,ARB,irbesartan,460
7,ACEi,ramipril,457
8,ACEi,moexipril,362
9,ARB,sacubitril-valsartan (24mg-26mg),310


**Descriptive Summary**

As a sanity check, drug names were collapsed to the active ingredient level. The majority of early RAAS exposure consisted of a small number of commonly prescribed agents, most notably lisinopril among ACE inhibitors and losartan/valsartan among ARBs. This distribution is consistent with expected inpatient prescribing patterns.

### 4.4 Missingness of Prescription Start Times

In [63]:
query="""
SELECT
  COUNT(*) AS n_total,
  SUM(CASE WHEN starttime IS NULL THEN 1 ELSE 0 END) AS n_starttime_null,
  SUM(CASE WHEN starttime IS NULL THEN 1 ELSE 0 END) * 100.0 / COUNT(*) AS pct_starttime_null
FROM `physionet-data.mimiciv_3_1_hosp.prescriptions`
WHERE hadm_id IS NOT NULL
  AND drug IS NOT NULL;
"""
client.query(query).to_dataframe()

,n_total,n_starttime_null,pct_starttime_null
0,20292611,21890,0.107872


**Descriptive Summary**

Medication start time was available for nearly all inpatient prescription records with a valid admission identifier and drug name. Only 0.11% of records had missing start times, indicating that missing temporal information is rare in the prescriptions table.

## 5. Outputs and Downstream Use

This notebook produces and validates admission-level tables in BigQuery that are used as fixed inputs for all downstream analyses.

**Outputs**

As a result of executing the SQL scripts in this notebook, the following tables are materialized in the
mimic-iv-portfolio.nonicu_raas dataset:

- **exposure_raas_early**
An admission-level table encoding early RAAS inhibitor exposure, defined as any ACE inhibitor or ARB prescription initiated within 24 hours after hospital admission. Exposure variables are stored as binary indicators (acei_early, arb_early, raas_both_early, raas_any_early).

- **analysis_dataset**
A unified, HADM-level analytic dataset created by left-joining the non-ICU admissions cohort with the early RAAS exposure table on (subject_id, hadm_id).
Each row corresponds to a single hospital admission, with exposure indicators explicitly coded for all admissions using zero-filling for non-exposed cases.

These tables are constructed entirely via SQL and are treated as finalized outputs of the data preparation stage.

**Downstream Use**

The analysis_dataset table serves as the sole input for subsequent notebooks, including:

- Baseline characteristic summaries (03b)
- Unadjusted outcome summaries and crude analyses (04a)
- Multivariable and adjusted outcome modeling (04b)

Downstream notebooks load this table in read-only mode and do not modify cohort definitions or exposure construction. This ensures that all descriptive and inferential analyses are based on a consistent and reproducible analytic dataset.